In [33]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## Empirical Practice of Difference-in-Differences 

### Callaway and Sant'Anna(2020)

To get more familiar with the recent methodological studies in dif-in-diff(DID) methods, I attempt to replicate [Callaway and Sant'Anna(2020)](https://pedrohcgs.github.io/files/Callaway_SantAnna_2020.pdf)(Hereafter CS). CS proposes a DID estimator for research designs with multiple time periods and units particularly when units are being treated at staggered time periods. As it is well-known causal inference method, a standard did design includes only two time periods and two units. For example please see the famous example of Card and Kruger Wage Study(1994) in 2x2 table below.

   |      | Pre | Post | Difference |
   | --- | --- | --- | --- |
   |New Jersey     |  20.44 | 21.03 | 0.59 |
   |Pennsylvania     |  23.33 | 21.17 | - 2.16 |
   |**Difference**      | - 2.89 |  -0.14 | 2.75 |

A standard 2x2 DID method is not applicable for cases such as Medicaid expansion. The Affordable Care Act (ACA) mandated to expand the elibility of Medicaid so the number of low-income Americans with health insurance increases. When Medicaid expansion took effect in 2014, 27 seven states adopted the expansion that year. As of 2021 the total number of states with Medicaid expansion is 39. Only 12 states have not adopted Medicaid expansion yet so far. Different expansion year makes the data perfect for a DID analysis with multiple time periods and groups. 

I apply CS method in this post using Medicaid expansion data which I extracted from American Community Survey (ACS) and [Kaiser Family Foundation website](https://www.kff.org/medicaid/issue-brief/status-of-state-medicaid-expansion-decisions-interactive-map/). The data includes median income level, uninsured and unemployment rate at the county level from 2011 to 2019. For the sake of transparency, I am sharing the data at [the this link](https://raw.githubusercontent.com/mehmet-sari/Exercises_Applications/main/medicaidexpansion.csv).

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

#pd.set_option('display.max_rows', 500)
%load_ext rpy2.ipython

#### Summary of the Data
The next two outputs show the data and summary statistics of uninsured rate, median income level, and unemployment rate.

In [2]:
df = pd.read_csv("output.csv", encoding='latin-1')
df['AdoptedYear'] = df['AdoptedYear'].replace([2020, 2021],0)
df.head(10)

,State,id,unemployment,median_income,uninsured,year,County,MedicaidStatus,AdoptedYear,SID
0,Alabama,0500000US01003,5.6,50900,12.0,2011,Baldwin County,Not Adopted,0,1
1,Alabama,0500000US01015,7.5,39037,15.6,2011,Calhoun County,Not Adopted,0,2
2,Alabama,0500000US01043,5.5,40054,12.6,2011,Cullman County,Not Adopted,0,3
3,Alabama,0500000US01049,7.1,36541,19.6,2011,DeKalb County,Not Adopted,0,4
4,Alabama,0500000US01051,6.5,57405,10.6,2011,Elmore County,Not Adopted,0,5
5,Alabama,0500000US01055,4.8,33313,14.4,2011,Etowah County,Not Adopted,0,6
6,Alabama,0500000US01069,5.2,40336,13.9,2011,Houston County,Not Adopted,0,7
7,Alabama,0500000US01073,7.2,41976,13.3,2011,Jefferson County,Not Adopted,0,8
8,Alabama,0500000US01077,3.1,40121,11.0,2011,Lauderdale County,Not Adopted,0,9
9,Alabama,0500000US01081,6.4,42965,12.6,2011,Lee County,Not Adopted,0,10


In [3]:
print('Summary Statistics')

df[["uninsured", "unemployment", "median_income"]].describe()

Summary Statistics


,uninsured,unemployment,median_income
count,7380.000000,7341.000000,7380.000000
mean,10.381260,6.009767,56981.417344
std,5.094168,2.350090,15740.901841
min,1.400000,1.000000,24945.000000
25%,6.400000,4.400000,45894.750000
50%,9.600000,5.600000,53360.500000
75%,13.500000,7.200000,64381.250000
max,37.900000,21.700000,151800.000000


For the first step, I will look at the impact of Medicaid expansion on uninsured rate at the county level without pre-treatment covariates.

Briefly, the identification CS method relies on parallel trends assumption after conditioning on pre-treatment covariates. Once there is no covariates in the analysis and the comparison group is 'never treated' group, CS proposes the following causal parameter of interest (equation 2.8 in their paper).

\begin{gather*}
ATT^{nev}_{unc} (g, t) = E[Y_{t} - Y_{g  - 1}|G_{g} = 1] - E[Y_{t} - Y_{g - 1}|C = 1]
\end{gather*}  

$ATT^{nev}_{unc} (g, t)$ compares the outcome evolution of group $g$, which are the units first treated at the time $g$, between the period of time $t$ and $g$-1 with the outcome evolution of comparison group, which are never treated group, between the same time period. The outcome evolution of comparison group provides the parallel trends in case the treated group would not experienced the effect of treatment. The parameter calculates the treatment effect for group $g$ at time $t$ and it is called group-time average treatment effect.

To give an example, let's calculate $ATT_{g, t}$ for group 2014 at time 2015 using the Medicaid expansion. Group of 2014 presents the states that adopted Medicaid expansion in 2014.

\begin{gather*}
ATT(2014,2015) = (Y_{2014,2015} - Y_{2014,2013}) - (Y_{C,2015} - Y_{C, 2013})
\end{gather*} 

Let's show this in 2x2 table:

   |      | Pre(2013) | Post(2015) | Difference |
   | --- | --- | --- | --- |
   |Adopted     |  Y(2014,2013) | Y(2014,2015) |  |
   |Non-Adopted     |  Y(C, 2013) | Y(C,2015)  | |
   |**Difference**      |  |   |    

The ATT parameter calculates the treatment effect by turning the data to a standard 2x2 DID setup with 2 groups (group of 2014 and never adopted group) and 2 time periods (2013 and 2015). Once ATT for group of 2014 is calculated for every year, it is possible to aggregate treatment effects in several ways to understand the treatment effect heterogeneity between groups.

Now I need to calculate ATT for each year for each group. For that, I need to create dummy variables for each group and treatment years for counties with Medicaid expansion.

In [4]:
## Create dummy variables for each cohort/group of treated counties.
dummies = pd.get_dummies(df, prefix='g', columns=['AdoptedYear'], drop_first= True)
df1 = pd.concat([dummies, df['AdoptedYear']],axis=1)

#Crate a dummy variable for treatment years for counties with medicaid expansion.
df1['treat'] = np.where((df1['AdoptedYear'] <= df1['year']), 1, 0)
df1.loc[df1['AdoptedYear'] == 0, 'treat'] = 0

In [5]:
## ATT estimation attempt to replicate what did package in R produces.

# function for dif-in-diff calculation.

def att(df,g,t):
    
    y11 = df[(df['MedicaidStatus'] == "Adopted") 
             & (df['AdoptedYear'] == g)
             & (df['year'] == t)]['uninsured'].mean()
             
    y10 = df[(df['MedicaidStatus'] == "Adopted") 
             & (df['AdoptedYear'] == g)
             & (df['year'] == g - 1)]['uninsured'].mean()
             
    y01 = df[(df['MedicaidStatus'] == "Not Adopted") 
             & (df['year'] == t)]['uninsured'].mean()    

    y00 = df[(df['MedicaidStatus'] == "Not Adopted") 
             & (df['year'] == g - 1)]['uninsured'].mean()
    
    did = (y11 - y10) - (y01 - y00)
    
    return did
## 
result = []
groups = df1['AdoptedYear'].unique()
times = df1['year'].unique()

for group in groups:
    if group > 0:
        for time in times:
            x = att(df, group, time)
            result.append((group,time,x))
            
att_estimate = pd.DataFrame(result)
att_estimate.columns = ['group','time', 'att']
att_estimate.sort_values(['group', 'time'])

,group,time,att
9,2014,2011,-0.120971
10,2014,2012,-0.037960
11,2014,2013,0.000000
12,2014,2014,-0.726021
13,2014,2015,-0.902026
14,2014,2016,-0.951459
15,2014,2017,-1.284129
16,2014,2018,-1.246134
17,2014,2019,-1.275390
0,2015,2011,-1.058772


Now I calculated group-time treatment affects for each group and time pair and I can calculate the aggregate ATT for each group. Before that I need to point that ATT(g,t) is zero once $t$<$g$ is. It means that ATT for time 2011, 2012, and 2013 are not considered in the aggregate ATT(g) for group of 2014. 

In [6]:
att_estimate[att_estimate['time'] >= att_estimate['group']].groupby(['group'])['att'].mean()

group
2014   -1.064193
2015   -0.847780
2016   -2.711116
2018   -0.962177
2019   -0.728915
Name: att, dtype: float64

So far, I calculate group-time treatment effects and aggregate ATTs for each group. Now, I would like to check my results with R package's result which is provided by CS. 

In [8]:
%%R
options(warn = -1)
install.packages("did", repos='http://cran.us.r-project.org', quiet=TRUE)
install.packages("ggplot2", repos='http://cran.us.r-project.org', quiet=TRUE)
install.packages("dplyr", repos='http://cran.us.r-project.org', quiet=TRUE)
library(did)
library(ggplot2)
library(dplyr)
df_medicaid <- read.csv('https://raw.githubusercontent.com/mehmet-sari/Exercises_Applications/main/medicaidexpansion.csv')
att <- att_gt(yname = "uninsured",  
              tname = "year",  
              idname = "SID", 
              gname = "AdoptedYear",  
              data = df_medicaid,  
              xformla = NULL,   
              est_method = "dr", 
              control_group = "nevertreated", 
              bstrap = TRUE,  
              biters = 1000,  
              print_details = FALSE)  
summary(att)

R[write to console]: Warning:
R[write to console]:  package 'did' is in use and will not be installed

R[write to console]: Warning:
R[write to console]:  package 'ggplot2' is in use and will not be installed

R[write to console]: Warning:
R[write to console]:  package 'dplyr' is in use and will not be installed




Call:
att_gt(yname = "uninsured", tname = "year", idname = "SID", gname = "AdoptedYear", 
    xformla = NULL, data = df_medicaid, control_group = "nevertreated", 
    bstrap = TRUE, biters = 1000, est_method = "dr", print_details = FALSE)

Reference: Callaway, Brantly and Pedro H.C. Sant'Anna.  "Difference-in-Differences with Multiple Time Periods." Forthcoming at the Journal of Econometrics <https://arxiv.org/abs/1803.09015>, 2020. 

Group-Time Average Treatment Effects:
 Group Time ATT(g,t) Std. Error [95% Simult.  Conf. Band]  
  2014 2012   0.1063     0.1603       -0.3665      0.5792  
  2014 2013   0.0539     0.1415       -0.3633      0.4711  
  2014 2014  -0.7857     0.1583       -1.2524     -0.3190 *
  2014 2015  -0.9508     0.1964       -1.5302     -0.3715 *
  2014 2016  -1.0373     0.2066       -1.6464     -0.4281 *
  2014 2017  -1.3097     0.1877       -1.8632     -0.7562 *
  2014 2018  -1.3181     0.2000       -1.9078     -0.7283 *
  2014 2019  -1.3658     0.2178       -2.0

It seems that the results from R package for CS method are not equal to the results I calculated manually. Numbers are close, trends are similar but somehow there is a slight difference between the results. Lastly I shall check the aggregate ATT results.

In [9]:
%%R 
att <-aggte(att, type = "group")
summary(att)


Call:
aggte(MP = att, type = "group")

Reference: Callaway, Brantly and Pedro H.C. Sant'Anna.  "Difference-in-Differences with Multiple Time Periods." Forthcoming at the Journal of Econometrics <https://arxiv.org/abs/1803.09015>, 2020. 


Overall ATT:  
     ATT Std. Error     [95%  Conf. Int.]  
 -1.1425     0.1396   -1.4161     -0.8688 *


Group Effects:
 group     ATT Std. Error [95% Simult.  Conf. Band]  
  2014 -1.1279     0.1708       -1.5501     -0.7056 *
  2015 -0.8332     0.1997       -1.3268     -0.3395 *
  2016 -2.7342     0.4132       -3.7557     -1.7127 *
  2018 -1.0161     0.3688       -1.9277     -0.1044 *
  2019 -0.7392     0.4578       -1.8708      0.3924  
---
Signif. codes: `*' confidence band does not cover 0

Control Group:  Never Treated,  Anticipation Periods:  0
Estimation Method:  Doubly Robust


The table below shows the results from my code and the results from R package side by side. Again the R package and my code produce similar but not exactly same results. 

|My Code |       | R Package |    |
| :--- | ---- | :--- | ---- |
group   |  ATT   |  group |  ATT |
2014 |  -1.064193 | 2014 | -1.1279 | 
2015 |  -0.847780 | 2015 | -0.8332 | 
2016 |  -2.711116 | 2016 | -2.7342 | 
2018 |  -0.962177 | 2018 | -1.0161 | 
2019 |  -0.728915 | 2019 | -0.7392 | 

In the next step, I will estimate the treatment effect of Medicaid expansion with covariates. For that, I will use the following parameter which includes propensity score estimation. 

\begin{gather*}
ATT(g,t)=[(\frac{G_{g}}{E\big[G_{g}\big]} - \frac{\frac{p_{g}(X)C}{1-pg(x}}{E\big[\frac{p_{g}(X)C}{1-p_{g}(X)}\big]})(Y_{t}-Y_{g-1})
\end{gather*}